## Resampling with rasterio (Local storage, COG file, COG driver)

In [ ]:
import argparse

import numpy as np
import rasterio
from rasterio.vrt import WarpedVRT

In [ ]:
def warp_resample(dataset, zoom=0):
    from common import target_extent

    te = target_extent[zoom]

    # Define source and target projection
    srcSRS = "EPSG:4326"
    dstSRS = "EPSG:3857"
    width = height = 256

    src = f"earthaccess_data/{dataset}.tif"

    with rasterio.open(src) as da:
        with WarpedVRT(da, src_crs=srcSRS, crs=dstSRS) as vrt:

            dst_window = vrt.window(*te)

            data = vrt.read(window=dst_window, out_shape=(height, width), masked=True)
            # Mask and fill array
            data = data.astype("float32", casting="unsafe")
            np.multiply(data, da.scales[0], out=data, casting="unsafe")
            np.add(data, da.offsets[0], out=data, casting="unsafe")
            return data

In [ ]:
%%time
if __name__ == "__main__":
    if "get_ipython" in dir():
        # Just call warp_resample if running as a Jupyter Notebook
        da = warp_resample("mursst")
    else:
        # Configure dataset via argpase if running via CLI
        parser = argparse.ArgumentParser(description="Set environment for the script.")
        parser.add_argument(
            "--dataset",
            default="mursst",
            help="Dataset to resample.",
            choices=["mursst"],
        )
        parser.add_argument(
            "--zoom",
            default=0,
            help="Zoom level for tile extent.",
        )
        user_args = parser.parse_args()
        da = warp_resample(user_args.dataset, int(user_args.zoom))